In [1]:
import pandas as pd
import numpy as np

## 1. Ufuncs: Index Preservation

Because Pandas is designed to work with **NumPy**, any **NumPy ufunc** will work on **Pandas Series** and **DataFrame objects.**

In [5]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, size = 4))
ser

0    6
1    3
2    7
3    4
dtype: int64

In [3]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)),
                  columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


If we apply a **NumPy ufunc** on **either of these objects**, the result will be another **Pandas object** with the indices preserved:

In [4]:
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

In [7]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


## 2. UFuncs: Index Alignment

#### 2.1 Index alignment in Series

In [8]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
                  'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                        'New York': 19651127}, name='population')

In [10]:
area

Alaska        1723337
Texas          695662
California     423967
Name: area, dtype: int64

In [11]:
population

California    38332521
Texas         26448193
New York      19651127
Name: population, dtype: int64

In [13]:
density = population / area
density

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

In [14]:
area.index | population.index

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

Any item for which **one or the other** does not have an **entry** is marked with **NaN**, or "Not a Number," which is how Pandas marks **missing data**.

In [15]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

If using **NaN values** is not the **desired behavior**, the **fill value** can be modified using **appropriate object methods** in place of the operators. For example, calling **A.add(B)** is equivalent to calling **A + B**, but **allows optional explicit specification** of the **fill value for any elements** in A or B that might be missing:

In [16]:
A.add(B, fill_value=0)

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

#### 2.2 Index alignment in DataFrame

In [17]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)),
                 columns=list('AB'))
A

,A,B
0,6,18
1,10,10


In [18]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)),
                 columns=list('BAC'))
B

,B,A,C
0,7,4,3
1,7,7,2
2,5,4,1


In [19]:
A + B

,A,B,C
0,10.0,25.0,NaN
1,17.0,17.0,NaN
2,NaN,NaN,NaN


Here we'll **fill with the mean of all values in A** (computed by first stacking the rows of A):

In [22]:
fill = A.stack().mean()
fill

11.0

In [23]:
A.add(B, fill_value=fill)

,A,B,C
0,10.0,25.0,14.0
1,17.0,17.0,13.0
2,15.0,16.0,12.0


#### 2.3 Ufuncs: Operations Between DataFrame and Series

When performing operations between a DataFrame and a Series, the **index and column alignment** is similarly **maintained**.

In [25]:
A = rng.randint(10, size=(3, 4))
A

array([[7, 5, 1, 4],
       [0, 9, 5, 8],
       [0, 9, 2, 6]])

In [26]:
A - A[0]

array([[ 0,  0,  0,  0],
       [-7,  4,  4,  4],
       [-7,  4,  1,  2]])

In Pandas, **the convention similarly** operates **row-wise** by default:

In [27]:
df = pd.DataFrame(A, columns=list('QRST'))
df

,Q,R,S,T
0,7,5,1,4
1,0,9,5,8
2,0,9,2,6


In [28]:
df - df.iloc[0]

,Q,R,S,T
0,0,0,0,0
1,-7,4,4,4
2,-7,4,1,2


If you would instead like to operate **column-wise**, you can use the **object methods** mentioned earlier, while specifying the **axis** keyword:

In [29]:
df.subtract(df['R'], axis=0)

,Q,R,S,T
0,2,0,-4,-1
1,-9,0,-4,-1
2,-9,0,-7,-3


Note that these **DataFrame/Series operations**, like the operations discussed above, will **automatically align indices** between the two elements:

In [30]:
halfrow = df.iloc[0, ::2]
halfrow

Q    7
S    1
Name: 0, dtype: int64

In [31]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,-7.0,NaN,4.0,NaN
2,-7.0,NaN,1.0,NaN
